In [ ]:
import os

from tqdm.auto import tqdm
from dotenv import load_dotenv

from ldap3 import Server, Connection, ALL, ObjectDef, AttrDef, Entry, Writer, SUBTREE, WritableEntry
from ldap3.utils.dn import parse_dn, safe_dn

In [ ]:
load_dotenv(dotenv_path=r"J:\RPA\.baseflow\.env", override=True)

In [ ]:
server = Server(
    host=os.environ["AD_LDAP_HOST"],
    port=int(os.environ["AD_LDAP_PORT"]),
    use_ssl=True,
    get_info=ALL,
)

conn = Connection(
    server,
    user=os.environ["AD_LDAP_USER"],
    password=os.environ["AD_LDAP_PASSWORD"],
    authentication="SIMPLE",
    auto_bind=True,
)

In [ ]:
conn.extend.standard.paged_search(
    search_base="DC=NYBORG,DC=DK",
    search_filter="(objectClass=person)",
    attributes=["*"],
    paged_size=1000,
    generator=False,
    get_operational_attributes=True,
)

ad_users = [{attr: entry[attr].value for attr in entry.entry_attributes} for entry in conn.entries]
len(ad_users)

In [ ]:
conn.search(
    search_base="OU=Nyborg,DC=NYBORG,DC=DK",
    search_filter="(objectClass=organizationalUnit)",
    search_scope=SUBTREE,
    attributes=["ou"]
)

ou_dns = [str(entry.entry_dn) for entry in conn.entries]
ou_dns

In [ ]:
obj_def = ObjectDef(["person"], conn)
attributes = ["objectSid", "distinguishedName", "userPrincipalName", "mail", "mailNickname", "proxyAddresses", "userAccountControl", "description"]

for attr in attributes:
    obj_def += AttrDef(attr)

conn.extend.standard.paged_search(
    search_base="OU=Sofd Nye brugere,DC=NYBORG,DC=DK",
    search_filter="(objectClass=person)",
    attributes=attributes,
    paged_size=1000,
    generator=False,
    get_operational_attributes=True,
)

entries: list[Entry] = conn.entries
len(entries)

In [ ]:
users = []
for entry in entries:

    user = {}
    for key, v in entry._state.attributes.items():
        user[key] = v.value if v.definition.single_value else v.values

    # raise if no description or description is more than one value
    if not user.get("description") or len(user["description"]) != 1:
        raise ValueError(f'User {user["userPrincipalName"]!r} has invalid description: {user["description"]!r}')

    # description is given as a list of strings
    # so we convert to single string for easier matching
    user["description"] = user["description"][0]

    # try direct OU match based on description
    target_dn = next((dn for dn in ou_dns if dn.replace("\\", "").startswith(f"OU={user["description"]},")), None)

    # if no direct match, find a user that contains the description and use that OU
    if not target_dn:

        print(f'No direct OU match for {user["userPrincipalName"]!r} with description {user["description"]!r}, trying indirect match...')

        target_ad_user = next((
            ad_user for ad_user in ad_users
            if (ad_user.get("description") == user["description"]) and (ad_user.get("userPrincipalName") != user["userPrincipalName"])
        ), None)

        if target_ad_user:

            # extract OU from target user's distinguishedName
            dn = parse_dn(target_ad_user["distinguishedName"])
            target_dn = [f"{attr}={value}" for attr, value, _ in dn[1:]]  # remove RDN of user itself
            target_dn = safe_dn(target_dn)

    user["target_dn"] = target_dn
    users += [user]

In [ ]:
print(f"Found {len(users)}/{len(entries)} users to move")
for user in users:
    print(f'({user["userPrincipalName"]!r}, {user["description"]!r}): {user["distinguishedName"]!r} → {user["target_dn"]!r}')

In [ ]:
modifications = []
for user in tqdm(users):

    upn = user["userPrincipalName"]

    print(f'Moving ({upn!r}, {user["description"]!r}) → {user["target_dn"]!r}')
    entry = next(e for e in entries if "userPrincipalName" in e and e["userPrincipalName"] == upn)
    entry: WritableEntry = entry.entry_writable(object_def=obj_def)
    entry.entry_move(destination_dn=user["target_dn"])

    print(entry.entry_changes)
    modifications += [entry.entry_changes]

    writer: Writer = entry.entry_cursor
    writer.commit()